![NYC Skyline](nyc.jpg)

Welcome to New York City, one of the most-visited cities in the world. There are many Airbnb listings in New York City to meet the high demand for temporary lodging for travelers, which can be anywhere between a few nights to many months. In this project, we will take a closer look at the New York Airbnb market by combining data from multiple file types like `.csv`, `.tsv`, and `.xlsx`.

Recall that **CSV**, **TSV**, and **Excel** files are three common formats for storing data. 
Three files containing data on 2019 Airbnb listings are available to you:

**data/airbnb_price.csv**
This is a CSV file containing data on Airbnb listing prices and locations.
- **`listing_id`**: unique identifier of listing
- **`price`**: nightly listing price in USD
- **`nbhood_full`**: name of borough and neighborhood where listing is located

**data/airbnb_room_type.xlsx**
This is an Excel file containing data on Airbnb listing descriptions and room types.
- **`listing_id`**: unique identifier of listing
- **`description`**: listing description
- **`room_type`**: Airbnb has three types of rooms: shared rooms, private rooms, and entire homes/apartments

**data/airbnb_last_review.tsv**
This is a TSV file containing data on Airbnb host names and review dates.
- **`listing_id`**: unique identifier of listing
- **`host_name`**: name of listing host
- **`last_review`**: date when the listing was last reviewed

In [116]:
# Import necessary packages
import pandas as pd
import numpy as np

In [117]:
df_prices = pd.read_csv('data/airbnb_price.csv')
df_prices.head()

,listing_id,price,nbhood_full
0,2595,225 dollars,"Manhattan, Midtown"
1,3831,89 dollars,"Brooklyn, Clinton Hill"
2,5099,200 dollars,"Manhattan, Murray Hill"
3,5178,79 dollars,"Manhattan, Hell's Kitchen"
4,5238,150 dollars,"Manhattan, Chinatown"


In [118]:
df_review = pd.read_csv('data/airbnb_last_review.tsv', sep='\t')
df_review.head()

,listing_id,host_name,last_review
0,2595,Jennifer,May 21 2019
1,3831,LisaRoxanne,July 05 2019
2,5099,Chris,June 22 2019
3,5178,Shunichi,June 24 2019
4,5238,Ben,June 09 2019


In [119]:
airbnb_room_type = pd.read_excel('data/airbnb_room_type.xlsx')
airbnb_room_type.head()

,listing_id,description,room_type
0,2595,Skylit Midtown Castle,Entire home/apt
1,3831,Cozy Entire Floor of Brownstone,Entire home/apt
2,5099,Large Cozy 1 BR Apartment In Midtown East,Entire home/apt
3,5178,Large Furnished Room Near B'way,private room
4,5238,Cute & Cozy Lower East Side 1 bdrm,Entire home/apt


In [120]:
df_aux = pd.merge(df_prices, df_review, on='listing_id', how='left')
df_airbnb = pd.merge(df_aux, airbnb_room_type, on='listing_id', how='left')
df_airbnb.head()

,listing_id,price,nbhood_full,host_name,last_review,description,room_type
0,2595,225 dollars,"Manhattan, Midtown",Jennifer,May 21 2019,Skylit Midtown Castle,Entire home/apt
1,3831,89 dollars,"Brooklyn, Clinton Hill",LisaRoxanne,July 05 2019,Cozy Entire Floor of Brownstone,Entire home/apt
2,5099,200 dollars,"Manhattan, Murray Hill",Chris,June 22 2019,Large Cozy 1 BR Apartment In Midtown East,Entire home/apt
3,5178,79 dollars,"Manhattan, Hell's Kitchen",Shunichi,June 24 2019,Large Furnished Room Near B'way,private room
4,5238,150 dollars,"Manhattan, Chinatown",Ben,June 09 2019,Cute & Cozy Lower East Side 1 bdrm,Entire home/apt


In [121]:
df_airbnb.isnull().sum()

listing_id      0
price           0
nbhood_full     0
host_name       8
last_review     0
description    10
room_type       0
dtype: int64

In [122]:
df_airbnb['last_review'] = pd.to_datetime(df_airbnb['last_review'])
df_airbnb.sample(3)

,listing_id,price,nbhood_full,host_name,last_review,description,room_type
23884,34857004,125 dollars,"Manhattan, Lower East Side",Gordy,2019-06-23,Sunny 1BR in the Heart of the Lower East Side,Entire home/apt
20295,31542976,70 dollars,"Manhattan, Washington Heights",Valerie,2019-06-24,"Cozy, Bright Room in Uptown Manhattan",Private room
13517,23851659,45 dollars,"Manhattan, Harlem",Frederic,2019-06-30,Cozy bedroom in the heart of South Harlem,Private room


In [123]:
df_airbnb.last_review.isnull().sum()

np.int64(0)

In [124]:
most_recent_review = df_airbnb['last_review'].max().date()
earliest_review = df_airbnb['last_review'].min().date()

print(f'Most recent review: {most_recent_review}')
print(f'Earliest review: {earliest_review}')

Most recent review: 2019-07-09
Earliest review: 2019-01-01


In [125]:
df_airbnb['room_type'] = df_airbnb['room_type'].apply(lambda x: x.lower())
df_airbnb.room_type.value_counts()

room_type
entire home/apt    13266
private room       11356
shared room          587
Name: count, dtype: int64

In [126]:
quantity_private_room = df_airbnb.query('room_type == "private room"').count()[0]
quantity_private_room

C:\Users\franf\AppData\Local\Temp\ipykernel_24464\2311544033.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  quantity_private_room = df_airbnb.query('room_type == "private room"').count()[0]


np.int64(11356)

In [127]:
def remove_units(price):
    try:
        return float(price.replace(' dollars', '').strip())
    except:
        return None  # o np.nan si estás trabajando con pandas

df_airbnb['price'] = df_airbnb['price'].apply(remove_units) 
df_airbnb.head()

,listing_id,price,nbhood_full,host_name,last_review,description,room_type
0,2595,225.0,"Manhattan, Midtown",Jennifer,2019-05-21,Skylit Midtown Castle,entire home/apt
1,3831,89.0,"Brooklyn, Clinton Hill",LisaRoxanne,2019-07-05,Cozy Entire Floor of Brownstone,entire home/apt
2,5099,200.0,"Manhattan, Murray Hill",Chris,2019-06-22,Large Cozy 1 BR Apartment In Midtown East,entire home/apt
3,5178,79.0,"Manhattan, Hell's Kitchen",Shunichi,2019-06-24,Large Furnished Room Near B'way,private room
4,5238,150.0,"Manhattan, Chinatown",Ben,2019-06-09,Cute & Cozy Lower East Side 1 bdrm,entire home/apt


In [128]:
average_price = round(df_airbnb['price'].mean(),2)
average_price

np.float64(141.78)

In [129]:
review_dates = pd.DataFrame(
    {
        'first_reviewed': [earliest_review],
        'last_reviewed': [most_recent_review],
        'nb_private_rooms': [quantity_private_room],
        'avg_price': [average_price],
    }
)

review_dates.head()

,first_reviewed,last_reviewed,nb_private_rooms,avg_price
0,2019-01-01,2019-07-09,11356,141.78
